# Introduction to Deep Learning

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

You are given a dataset containing:
- a training set of m_train images labeled as cat (y=1) or non-cat (y=0)
- a test set of m_test images labeled as cat or non-cat
- each image is of shape `(num_px, num_px, 3)` where 3 is for the 3 channels (RGB). Thus, each image is square (height = num_px) and (width = num_px).

You will build a simple image-recognition algorithm that can correctly classify pictures as cat or non-cat.

Let's get more familiar with the dataset. Load the data by running the following code.

In [ ]:
def loadDataset():
    trainDataset = h5py.File("datasets/train_catvnoncat.h5", "r")
    trainDatasetX = np.array(trainDataset["train_set_x"][:])
    trainDatasetY = np.array(trainDataset["train_set_y"][:])
    
    testDataset = h5py.File("datasets/test_catvnoncat.h5")
    testDatasetX = np.array(testDataset["test_set_x"][:])
    testDatasetY = np.array(testDataset["test_set_y"][:])

    classes = np.array(trainDataset["list_classes"][:])

    trainDatasetY = trainDatasetY.reshape((1, trainDatasetY.shape[0]))
    testDatasetY = testDatasetY.reshape((1, testDatasetY.shape[0]))
    return trainDatasetX, trainDatasetY, testDatasetX, testDatasetY, classes

trainDatasetX, trainDatasetY, testDatasetX, testDatasetY, classes = loadDataset()
print(trainDatasetX.shape, trainDatasetY.shape)

Each line of your train_set_x_orig and test_set_x_orig is an array representing an image. You can visualize an example by running the following code. Feel free also to change the `index` value and re-run to see other images. 

In [ ]:
index = 10
plt.imshow(trainDatasetX[index])
plt.show()
print ("y = " + str(trainDatasetY[0, index]) + ", it's a '" + classes[np.squeeze(trainDatasetY[:, index])].decode("utf-8") +  "' picture.")

Find the values for:
- m_train (number of training examples)
- m_test (number of test examples)
- num_px (= height = width of a training image)  
Remember that `train_set_x_orig` is a numpy-array of shape (m_train, num_px, num_px, 3). For instance, you can access `m_train` by writing `train_set_x_orig.shape[0]`.

In [ ]:
m_train = None
m_test = None
num_px = None

# let's print some usefull imformation
print("Number of training examples: ", m_train)
print("Number of testing examples: ", m_test)
print("Height of image: ", num_px, "px")
print("Shape of image: ", trainDatasetX[0].shape)
print("Shape of training dataset X: ", trainDatasetX.shape)
print("Shape of training dataset Y: ", trainDatasetY.shape)

For convenience, you should now reshape images of shape (num_px, num_px, 3) in a numpy-array of shape (num_px $*$ num_px $*$ 3, 1). After this, our training (and test) dataset is a numpy-array where each column represents a flattened image. There should be m_train (respectively m_test) columns.

A trick when you want to flatten a matrix X of shape (a,b,c,d) to a matrix X_flatten of shape (b * c * d, a) is to use: 
```
X_flatten = X.reshape(X.shape[0], -1).T
```

In [ ]:
XTrainFlatten = None
XTestFlatten = None
print("Shape of XFlatten: ", XTrainFlatten.shape)
print("Shape of XTestFlatten: ", XTestFlatten.shape)

To represent color images, the red, green and blue channels (RGB) must be specified for each pixel, and so the pixel value is actually a vector of three numbers ranging from 0 to 255.

One common preprocessing step in machine learning is to center and standardize your dataset, meaning that you substract the mean of the whole numpy array from each example, and then divide each example by the standard deviation of the whole numpy array. But for picture datasets, it is simpler and more convenient and works almost as well to just divide every row of the dataset by 255 (the maximum value of a pixel channel).

In [ ]:
XTrainFlatten = XTrainFlatten / 255
XTestFlatten = XTestFlatten / 255

Sigmoid Function

In [ ]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

Initialize Parameters

In [ ]:
def initializeParameters(dim):
    w = None
    b = None
    return w, b

## Logistic Regression 
#### Forward Propagation
$$
Z = W^T X + b  \\
A = sigmoid(X) \\
$$
#### Calculate Cost
$$
J = -\frac{1}{m}\sum_{i=1}^{m}y^{(i)}\log(a^{(i)})+(1-y^{(i)})\log(1-a^{(i)})
$$
#### Backward Propagation
$$ \partial W = \frac{\partial J}{\partial W} = \frac{1}{m}X(A-Y)^T\tag{7}$$
$$ \partial b = \frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)}-y^{(i)})\tag{8}$$
#### Parameter Updation
$$ W = W - \alpha \text{ } \partial W $$
$$ b = b - \alpha \text{ } \partial b $$

Let's implements forward propagation, cost calculation and backward propagation in one function

In [ ]:
def propagate(w, b, X, Y):
    m = None

    #forward prop
    Z = None
    A = None

    #cost calculation
    cost = None

    #backward prop
    db = None
    dw = None
    grad = {"dw": dw,
            "db": db}
    
    return grad, cost

Implement gradient decent for training our weights

In [ ]:
def train_model(X, Y, numIterations, learningRate, printCost = False):
    nx = X.shape[0]
    m = X.shape[1]
    w, b = initializeParameters(nx)
    costs = []

    for i in range(numIterations):
        #forward propagation, cost calculation and backward propagation
        grad, cost = propagate(w, b, X, Y)
        costs.append(cost)
        #update weights
        w -= None
        b -= None

        if(printCost and i%100==0):
            print(f"Cost after {i} iterations: {cost}")
    
    params = {"w": w,
              "b": b}
    
    return params, costs

In [ ]:
params, costs = train_model(XTrainFlatten, trainDatasetY, numIterations = 2000, learningRate = 0.005, printCost = True)

Let's see how cost decrease with iterations

In [ ]:
plt.plot(costs)
plt.show()

Let's Predict out model's output on train and test images and and see how our model performed

In [ ]:
def predict(params, X, Y):
    w = params["w"]
    b = params["b"]
    Z = w.T.dot(X) + b
    A = sigmoid(Z)
    Y_Prediction = (A > 0.5) * 1.0
    return Y_Prediction

#checking accuracy
trainPrediction = predict(params, XTrainFlatten, trainDatasetY)
testPrediction = predict(params, XTestFlatten, testDatasetY)
print(f"train accuracy: {100-np.mean(np.abs(trainPrediction-trainDatasetY))*100}")
print(f"test accuracy: {100-np.mean(np.abs(testPrediction-testDatasetY))*100}")